### Example Prompts:
- Blood & Water  
- Ganglands  
- Kota Factory  
- Jailbirds New Orleans  



In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# loading dataset
url = "https://github.com/shaswat373/Movie-Recommendation/raw/main/dataset.zip"
path = "dataset.zip"
extract_to = "movies"

import urllib.request, zipfile, os
if not os.path.exists("netflix_data.csv"):
    print("Downloading dataset...")
    urllib.request.urlretrieve(url, path)
    with zipfile.ZipFile(path, 'r') as zip_ref:
        zip_ref.extractall()
    print("Dataset ready.")

df = pd.read_csv("netflix_data.csv")
df = df.dropna(subset=['title', 'description', 'listed_in'])  # ensure required fields
df['content'] = df['description'] + " " + df['listed_in']  # combine description and genres
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df['content'])

cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
title_to_index = pd.Series(df.index, index=df['title'].str.lower())

# recommendation function
def recommend_movies(title, n=5):
    title = title.strip().lower()
    if title not in title_to_index:
        print(f"\nMovie '{title}' not found in database. Please check the spelling.")
        return

    idx = title_to_index[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:n+1]

    print(f"\nBecause you liked '{df['title'][idx]}', you might also enjoy:\n")
    for i, (movie_idx, score) in enumerate(sim_scores, 1):
        print(f"{i}. {df['title'][movie_idx]}")

# user input
print("Movie Recommendation System")
movie = input("Enter a movie title: ")
recommend_movies(movie)

